# prompt_interaction_base.ipynb
> A notebook for formulating prompts and prompting

In this notebook, we create some base functionality for creating prompts and getting answers for the LLMs in a simplified, unified way.

:::{.callout-caution}
These notebooks are development notebooks, meaning that they are meant to be run locally or somewhere that supports navigating a full repository (in other words, not Google Colab unless you clone the entire repository to drive and then mount the Drive-Repository.) However, it is expected if you're able to do all of those steps, you're likely also able to figure out the required pip installs for development there.
:::


In [ ]:
#| default_exp PromptInteractionBase

In [ ]:
#| export
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.chains.base import Chain
from langchain.schema.messages import SystemMessage, HumanMessage
from langchain.prompts.chat import ChatPromptValue

from getpass import getpass

import os

In [ ]:
# imports for testing
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

## Model and Authentication Setup
Here, we create functionality to authenticate the user when needed specifically using OpenAI models. Additionally, we create the capacity to make LLMChains and other chains using one unified interface.

In [ ]:
#| export
def create_model(openai_mdl='gpt-3.5-turbo-16k', temperature=0.1, **chatopenai_kwargs):
    llm = ChatOpenAI(model_name = openai_mdl, temperature=temperature, **chatopenai_kwargs)

    return llm

In [ ]:
#| export
def set_openai_key():
    os.environ["OPENAI_API_KEY"] = getpass()

    return

**And now for a quick test of this functionality**

In [ ]:
set_openai_key()
assert os.environ["OPENAI_API_KEY"], "Either you didn't run set_openai_key or you haven't set it to something."

chat_mdl = create_model()
assert isinstance(chat_mdl, ChatOpenAI), "The default model type is currently ChatOpenAI. If that has changed, change this test."

## Create chat prompt templates
Here, we'll create a tutor prompt template to help us with self-study and quizzing, and help create the student messages.

In [ ]:
#| export
# Create system prompt template
SYSTEM_TUTOR_TEMPLATE = ("You are a world-class tutor helping students to perform better on oral and written exams though interactive experiences. " +
                         "When assessing and evaluating students, you always ask one question at a time, and wait for the student's response before " +
                         "providing them with feedback. Asking one question at a time, waiting for the student's response, and then commenting " +
                         "on the strengths and weaknesses of their responses (when appropriate) is what makes you such a sought-after, world-class tutor.")

# Create a human response template
HUMAN_RESPONSE_TEMPLATE = ("I'm trying to better understand the text provided below. {assessment_request} The learning objectives to be assessed are: " +
                           "{learning_objectives}. Although I may request more than one assessment question, you should " +
                           "only provide ONE question in you initial response. Do not include the answer in your response. " +
                           "If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional " +
                           "chances to respond until I get the correct choice. Explain why the correct choice is right. " +
                           "The text that you will base your questions on is as follows: {context}.")

HUMAN_RETRIEVER_RESPONSE_TEMPLATE = ("I want to master the topics based on the excerpts of the text below. Given the following extracted text from long documents, {assessment_request} The learning objectives to be assessed are: " +
                           "{learning_objectives}. Although I may request more than one assessment question, you should " +
                           "only provide ONE question in you initial response. Do not include the answer in your response. " +
                           "If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional " +
                           "chances to respond until I get the correct choice. Explain why the correct choice is right. " +
                           "The extracted text from long documents are as follows: {summaries}.")

def create_base_tutoring_prompt(system_prompt=None, human_prompt=None):

    #setup defaults using defined values
    if system_prompt == None:
        system_prompt = PromptTemplate(template = SYSTEM_TUTOR_TEMPLATE,
                          input_variables = [])
    
    if human_prompt==None:
        human_prompt = PromptTemplate(template = HUMAN_RESPONSE_TEMPLATE,
                           input_variables=['assessment_request', 'learning_objectives', 'context'])

    # Create prompt messages
    system_tutor_msg = SystemMessagePromptTemplate(prompt=system_prompt)
    human_tutor_msg = HumanMessagePromptTemplate(prompt= human_prompt)

    # Create ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_tutor_msg, human_tutor_msg])

    return chat_prompt

Now for a quick unit test for testing...

In [ ]:
chat_prompt = create_base_tutoring_prompt()
assert chat_prompt.messages[0].prompt.template == SYSTEM_TUTOR_TEMPLATE, "Did not set up the first chat_prompt to be SystemMessage"
assert chat_prompt.messages[1].prompt.template == HUMAN_RESPONSE_TEMPLATE, "Did not set up the second element of chat_prompt to be HumanMessage"

Now, let's define a function that allows us to set up default variables in case the user chooses not to pass something in.

In [ ]:
#| export
DEFAULT_ASSESSMENT_MSG = 'Please design a 5 question short answer quiz about the provided text.'
DEFAULT_LEARNING_OBJS_MSG = 'Identify and comprehend the important topics and underlying messages and connections within the text'

def get_tutoring_prompt(context, chat_template=None, assessment_request = None, learning_objectives = None, **kwargs):

    # set defaults
    if chat_template is None:
        chat_template = create_base_tutoring_prompt()
    else:
        if not all([prompt_var in chat_template.input_variables
                    for prompt_var in ['context', 'assessment_request', 'learning_objectives']]):
            raise KeyError('''It looks like you may have a custom chat_template. Either include context, assessment_request, and learning objectives
                           as input variables or create your own tutoring prompt.''')

    if assessment_request is None and 'assessment_request':
        assessment_request = DEFAULT_ASSESSMENT_MSG
    
    if learning_objectives is None:
        learning_objectives = DEFAULT_LEARNING_OBJS_MSG
    
    # compose final prompt
    tutoring_prompt = chat_template.format_prompt(context=context,
                                                assessment_request = assessment_request,
                                                learning_objectives = learning_objectives,
                                                **kwargs)
    
    return tutoring_prompt


**Another quick unit test...**

In [ ]:
# For defaults
res = get_tutoring_prompt('The dog was super pretty and cute').to_messages()
res

[SystemMessage(content="You are a world-class tutor helping students to perform better on oral and written exams though interactive experiences.\nWhen assessing and evaluating students, you always ask one question at a time, and wait for the student's response before providing them with feedback.\nAsking one question at a time, waiting for the student's response, and then commenting on the strengths and weaknesses of their responses (when appropriate)\nis what makes you such a sought-after, world-class tutor.", additional_kwargs={}),
 HumanMessage(content="I'm trying to better understand the text provided below. Please design a 5 question short answer quiz about the provided text. The learning objectives to be assessed are:\nIdentify and comprehend the important topics and underlying messages and connections within the text. Although I may request more than one assessment question, you should\nonly provide ONE question in you initial response. Do not include the answer in your response

Now, let's finally define how we can get the chat response from the model.

In [ ]:
#| export
def get_tutoring_answer(context, tutor_mdl, chat_template=None,
                        assessment_request=None,
                        learning_objectives=None,
                        return_dict=False,
                        call_kwargs=None,
                        input_kwargs=None):
    
    if call_kwargs is None:
        call_kwargs = {}
    if input_kwargs is None:
        input_kwargs = {}
    
    # set defaults
    if assessment_request is None:
            assessment_request = DEFAULT_ASSESSMENT_MSG
    if learning_objectives is None:
        learning_objectives = DEFAULT_LEARNING_OBJS_MSG
    
    common_inputs = {'assessment_request':assessment_request, 'learning_objectives':learning_objectives}
    
    # get answer based on interaction type
    if isinstance(tutor_mdl, ChatOpenAI):
        human_ask_prompt = get_tutoring_prompt(context, chat_template, assessment_request, learning_objectives)
        tutor_answer = tutor_mdl(human_ask_prompt.to_messages())

        if not return_dict:
            final_answer = tutor_answer.content
    
    elif isinstance(tutor_mdl, Chain):
        if isinstance(tutor_mdl, RetrievalQAWithSourcesChain):
            if 'question' not in input_kwargs.keys():
                common_inputs['question'] = assessment_request
            final_inputs = {**common_inputs, **input_kwargs}
            
        elif isinstance(tutor_mdl, ConversationalRetrievalChain):
            if 'question' not in input_kwargs.keys() or not input_kwargs['question']:
                system_human_question = get_tutoring_prompt(context, chat_template, assessment_request, learning_objectives)
                question = [msg for msg in system_human_question.messages if isinstance(msg, HumanMessage)]
                question = question[0].content if question else question.to_string()
                input_kwargs['question'] = question

            if 'chat_history' not in input_kwargs.keys():
                input_kwargs['chat_history'] = []
            final_inputs = {**input_kwargs}
        else:
            common_inputs['context'] = context
            final_inputs = {**common_inputs, **input_kwargs}
         
        # get answer
        tutor_answer = tutor_mdl(final_inputs, **call_kwargs)
        final_answer = tutor_answer

        if not return_dict:
            final_answer = final_answer['answer']
    
    else:
        raise NotImplementedError(f"tutor_mdl of type {type(tutor_mdl)} is not supported.")

    return final_answer

In [ ]:
#| export

DEFAULT_CONDENSE_PROMPT_TEMPLATE = ("Given the following conversation and a follow up message, combine the chat history and follow up message into " +
    "a concise standalone message in the original language. The created standalone message should: 1) Ensure that the overarching task and characteristics of the interaction are always contained in the summary. For example, "+
    "if the user asks that 5 questions should be asked, this instruction should always be in the summary until the 5 question task is completed. Additionally, if the instructions contain information "+
    "about your role - e.g., that you are an exceptional Socratic tutor and great at helping students better understand information, then this should also always be in the summary message, "+
    "2) concisely summarize the key points of the history of the conversation while taking into account the goal and of the follow up message, "+
    "3) provide suitable context from the conversation to inform the follow up message, and 4) be written in a way that emphasizes the new follow up message, " +
    "but again always keeping in mind any overarching tasks (until they are completed) and characteristics of the interaction. " + 
                                    "\n\nChat History:\n{chat_history}\nFollow Up message: {question}\nStandalone message:")

DEFAULT_QUESTION_PROMPT_TEMPLATE  = ("Use the following portion of a long document to see if any of the text is relevant to creating a response to the question." +
                                     "\nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:")

DEFAULT_COMBINE_PROMPT_TEMPLATE = ("Given the following extracted parts of a long document and the given prompt, create a final answer with references ('SOURCES'). "+
                                   "If you don't have a response, just say that you are unable to come up with a response. "+
                                   "\nSOURCES:\n\nQUESTION: {question}\n=========\n{summaries}\n=========\nFINAL ANSWER:'")

def create_tutor_mdl_chain(kind='llm', mdl=None, prompt_template = None, **kwargs):
    
    #Validate parameters
    if mdl is None:
        mdl = create_model()
    kind = kind.lower()
    
    #Create model chain
    if kind == 'llm':
        if prompt_template is None:
            prompt_template = create_base_tutoring_prompt()
        mdl_chain = LLMChain(llm=mdl, prompt=prompt_template, **kwargs)
    elif kind == 'conversational':
        if prompt_template is None:
            prompt_template = PromptTemplate.from_template(DEFAULT_CONDENSE_PROMPT_TEMPLATE)
            prompt_template = create_base_tutoring_prompt(human_prompt=prompt_template)
        mdl_chain = ConversationalRetrievalChain.from_llm(mdl, condense_question_prompt = prompt_template, **kwargs)
        #mdl_chain = ConversationalRetrievalChain.from_llm(mdl, **kwargs)
    elif kind == 'retrieval_qa':
        if prompt_template is None:

            #Create custom human prompt to take in summaries
            human_prompt = PromptTemplate(template = HUMAN_RETRIEVER_RESPONSE_TEMPLATE,
                           input_variables=['assessment_request', 'learning_objectives', 'summaries'])
            prompt_template = create_base_tutoring_prompt(human_prompt=human_prompt)
            
        #Create the combination prompt and model
        question_template = PromptTemplate.from_template(DEFAULT_QUESTION_PROMPT_TEMPLATE)
        #mdl_chain = RetrievalQAWithSourcesChain.from_llm(llm=mdl, question_prompt=question_template, combine_prompt = prompt_template, **kwargs)
        mdl_chain = RetrievalQAWithSourcesChain.from_chain_type(llm=mdl, **kwargs)
    else:
        raise NotImplementedError(f"Model kind {kind} not implemented")
    
    return mdl_chain

**Another brief test of behavior of these functions**

In [ ]:
res = get_tutoring_answer('The dog is super cute', chat_mdl)
print(res)

**Get the simplest type of response from the model**

In [ ]:
#| export
def get_direct_answer(mdl, human_prompt=' ', system_prompt=' ', return_dict=False):

    # get answer based on interaction type
    if isinstance(mdl, ChatOpenAI):
        human_ask_prompt = ChatPromptValue(messages=[HumanMessage(content=human_prompt),
                                                       SystemMessage(content=system_prompt)])
        mdl_answer = mdl(human_ask_prompt.to_messages())

        if not return_dict:
            final_answer = mdl_answer.content

    else:
        raise NotImplementedError(f"mdl of type {type(mdl)} is not supported.")

    return final_answer

Test this response type

In [ ]:
test_system = ''
test_human = 'You are an ultra helpful tutor. Whenever the user says hi, you should say hi back and introduce yourself.'
res = get_direct_answer(chat_mdl, test_human, test_system)
res

"Hi there! Thank you for the kind words. I'm an ultra helpful tutor here to assist you with any questions or problems you may have. How can I help you today?"

### Validate LLM Chain

In [ ]:
# Try llm model chain, making sure we've set the API key
llm_chain_test = create_tutor_mdl_chain('llm')
res = llm_chain_test.run({'context':'some context', 'assessment_request':'some assessment', 'learning_objectives':'some prompt'})

In [ ]:
# Verify information about the cell above
print(type(llm_chain_test))
print(res)

# unit tests
assert isinstance(llm_chain_test, LLMChain), 'the output of llm create_tutor_mdl_chain should be an instance of LLMChain'
assert isinstance(res, str), 'the output of running the llm chain should be of type string.'

<class 'langchain.chains.llm.LLMChain'>


'Sure, I can help you with that. Please provide me with the specific text that you would like me to base my questions on.'

Now, we'll try this with just the default function to run things...

In [ ]:
res = get_tutoring_answer(context='some context', tutor_mdl = llm_chain_test)
res

{'context': 'some context',
 'assessment_request': 'Please design a 5 question short answer quiz about the provided text.',
 'learning_objectives': 'Identify and comprehend the important topics and underlying messages and connections within the text',
 'text': 'Question 1: What are the main topics discussed in the text?\n\n(Note: Please provide your answer and I will provide feedback accordingly.)'}

OK, this base functionality is looking good. Now, let's look at the conversational one.


### Validate ConversationalRetrievalChain

Here, we'll create a small database of documents and use the conversational retrieval chain to interact with it.

In [ ]:
set_openai_key()

In [ ]:
# read file text
with open('../roadnottaken.txt', 'r') as f:
    long_text = f.read()

embeddings = OpenAIEmbeddings()
db = Chroma.from_texts([long_text], embeddings)
retriever = db.as_retriever()

In [ ]:
# Check to see that the chroma db is working correctly
retriever.get_relevant_documents('The dog is super cute')

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(page_content='\ufeffTwo roads diverged in a yellow wood,\nAnd sorry I could not travel both\nAnd be one traveler, long I stood\nAnd looked down one as far as I could\nTo where it bent in the undergrowth;\n\nThen took the other, as just as fair,\nAnd having perhaps the better claim,\nBecause it was grassy and wanted wear;\nThough as for that the passing there\nHad worn them really about the same,\n\nAnd both that morning equally lay\nIn leaves no step had trodden black.\nOh, I kept the first for another day!\nYet knowing how way leads on to way,\nI doubted if I should ever come back.\n\nI shall be telling this with a sigh\nSomewhere ages and ages hence:\nTwo roads diverged in a wood, and IэI took the one less traveled by,\nAnd that has made all the difference.\n')]

In [ ]:
# Now, we'll create the conversational retrieval chain
llm_chain_test = create_tutor_mdl_chain('conversational', retriever=retriever)
res = get_tutoring_answer(context='some context', tutor_mdl = llm_chain_test,
                          assessment_request = 'I want to study the road not taken to better understand metaphorical language')
print(res)

Based on the given text, here is an assessment question for you:

What is the speaker's reason for choosing the road "less traveled by"?

A) The road was more scenic and beautiful.
B) The road was less worn and had not been traveled as much.
C) The speaker wanted to take a different path than others.
D) The speaker was unsure if they would ever come back to this road.

Please choose one option and I will provide feedback on your answer.


Great. Looks like conversational chain works.

### Validate Conversational Retrieval Chain with Chat History
Now, we'll demonstrate how to use this with Chat History

In [ ]:
conversation_history = []
hist_res = get_tutoring_answer(context='some context', tutor_mdl = llm_chain_test,\
                               assessment_request = 'I want to study the road not taken to better understand metaphorical language',
                               return_dict=True,
                               input_kwargs={'chat_history':conversation_history})

Get here


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [ ]:
#show response
hist_res

{'chat_history': [],
 'question': "I'm trying to better understand the text provided below. I want to study the road not taken to better understand metaphorical language The learning objectives to be assessed are: Identify and comprehend the important topics and underlying messages and connections within the text. Although I may request more than one assessment question, you should only provide ONE question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional chances to respond until I get the correct choice. Explain why the correct choice is right. The text that you will base your questions on is as follows: some context.",
 'answer': 'Based on the given text, here is an assessment question for you:\n\nWhat is the speaker\'s reason for choosing the road "less traveled by"?\n\nA) It was more scenic and beautiful.\nB) It was less worn and less traveled.\nC) It ha

In [ ]:
# update conversation history and add answer
conversation_history.append((hist_res['question'], hist_res['answer']))

# create demo answer based on previous statement
demo_answer = 'The correct answer is B - it was less worn and less traveled. As I understand, this is the entire point of the poem.'

# send demo answer to model and get response
hist_res2 = get_tutoring_answer(context='some context', tutor_mdl = llm_chain_test,
                               return_dict=True,
                               input_kwargs={'chat_history':conversation_history,
                                             'question':demo_answer})
hist_res2

Get here


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'chat_history': [("I'm trying to better understand the text provided below. I want to study the road not taken to better understand metaphorical language The learning objectives to be assessed are: Identify and comprehend the important topics and underlying messages and connections within the text. Although I may request more than one assessment question, you should only provide ONE question in you initial response. Do not include the answer in your response. If I get an answer wrong, provide me with an explanation of why it was incorrect, and then give me additional chances to respond until I get the correct choice. Explain why the correct choice is right. The text that you will base your questions on is as follows: some context.",
   'Based on the given text, here is an assessment question for you:\n\nWhat is the speaker\'s reason for choosing the road "less traveled by"?\n\nA) It was more scenic and beautiful.\nB) It was less worn and less traveled.\nC) It had a better claim and wa

Wonderful. This looks great!

## Helper code for Conversation History

In [ ]:
#| export
def update_conversation_history(chat_history, question, answer):
    chat_history.append((question, answer))

    return chat_history 

In [ ]:
# Quick test
demo_history = []
update_conversation_history(demo_history, 'What is the answer to the question?', 'The answer is A')
update_conversation_history(demo_history, 'What is the answer to the new question?', 'The answer is B')
demo_history

[('What is the answer to the question?', 'The answer is A'),
 ('What is the answer to the new question?', 'The answer is B')]

This looks great!